In [6]:
!pip install torch transformers flask beautifulsoup4 requests nltk

In [27]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [28]:
from transformers import BertForSequenceClassification, BertTokenizer

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
import numpy as np
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")  # Replace with your own MongoDB connection string
db = client.questions_database  # Use the questions_database database in MongoDB

# Access the collection
questions_collection = db['questions']

# Find all documents in the collection
documents = questions_collection.find()

# Initialize lists to store texts and labels
input_texts = []
output_labels = []

# Iterate through each document
for document in documents:
    # Access the 'questions' array
    questions_array = document['questions']
    
    # Iterate through each question in the array
    for question_info in questions_array:
        # Access the 'question' field
        question_text = question_info['question']
        answers_text = question_info['answer']
        
        # Append the question to the 'texts' list if it's not already there
        if question_text not in input_texts:
            input_texts.append(question_text)
            # Assign a label based on some condition (binary classification)
            if some_condition_based_on_question(question_text):
                output_labels.append(1)  # Intent is positive
            else:
                output_labels.append(0)  # Intent is negative

#  Tokenize input texts
tokenized_inputs = tokenizer(input_texts, truncation=True, padding=True, return_tensors='pt')

#  Prepare input and label tensors
input_ids = tokenized_inputs['input_ids']
attention_mask = tokenized_inputs['attention_mask']
labels = torch.tensor(output_labels)

#  Set up training arguments and trainer
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=torch.utils.data.TensorDataset(input_ids, attention_mask, labels),
)

#  Train the model
trainer.train()


ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided []